$f(x,y) = - \left(\dfrac{3}{(x^2+ 8)} + \dfrac{5}{(y^2+ 8)}\right)$


#Двуслойная нейросеть. Вариант 29 (PReLU,Гиперболический тангенс,SGD) Едленко 4к. 6г.

In [223]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from matplotlib import cm
import time
import math
import pickle
import os.path

from keras.utils import to_categorical
from keras import models
from keras import layers

In [224]:
def prelu(x, alpha):
    return np.where( x < 0, alpha * x, x)

def prelu_diff(x, alpha):
    return np.where(x < 0, np.ones(x.shape) * alpha, np.ones(x.shape))

def sigmoid(x):
    return 1/(1 + np.exp(-x))

def sigmoid_diff(y):
    return y*(1 - y)

def tanh(x):
    return 2 * sigmoid(2*x) - 1

def tanh_diff(x):
    return 1 - tanh(x)**2



In [225]:
from numpy.lib import math
class NeuralNetwork_two:
    def __init__(self, x, y): #  объём датасета
        self.input      = x 
        self.y          = y 
        self.diam_1     = 8
        self.diam_2     = 6
        self.weights1   = np.random.rand(self.diam_1, self.input.shape[0]) 
        self.weights2   = np.random.rand(self.diam_2, self.diam_1) 
        self.weights3   = np.random.rand(1, self.diam_2) # 
        self.output     = np.zeros(self.y.shape) 
        self.alpha      = 0.3
        self.bias_1     = np.random.rand(self.diam_1, 1)
        self.bias_2     = np.random.rand(self.diam_2, 1)
        self.b          = np.random.rand()
        
    def feedforward(self):
        self.layer1 = prelu(np.dot(self.weights1, self.input) + self.bias_1, self.alpha) 
        self.layer2 = tanh(np.dot(self.weights2, self.layer1) + self.bias_2) 
        self.output = np.dot(self.weights3, self.layer2) + self.b
        
    def backprop(self):
        d_weights3 = 2*np.dot((self.y - self.output),self.layer2.T) 
        d_weights2 = 2*self.weights3.T*np.dot((self.y - self.output)*prelu_diff(self.layer2, self.alpha),self.layer1.T )
        d_weights1 = 2*np.dot(np.dot(self.weights3*self.weights2.T,(self.y - self.output)*prelu_diff(self.layer2, self.alpha))*prelu_diff(self.layer1, self.alpha),self.input.T)
        d_bias_2   = 2*self.weights3.T*np.dot(prelu_diff(self.layer2, self.alpha), (self.y - self.output).T)
        d_bias_1   = 2*np.dot(np.dot(self.weights3*self.weights2.T,prelu_diff(self.layer2, self.alpha))*prelu_diff(self.layer1, self.alpha), (self.y - self.output).T)
        d_b        = 2*(self.y - self.output).sum()


        self.weights1 += self.alpha * d_weights1/self.input.shape[1]
        self.weights2 += self.alpha * d_weights2/self.input.shape[1]
        self.weights3 += self.alpha * d_weights3/self.input.shape[1]
        self.bias_1   += self.alpha * d_bias_1/self.input.shape[1]
        self.bias_2   += self.alpha * d_bias_2/self.input.shape[1]
        self.b        += self.alpha * d_b/self.input.shape[1]

    def test(self, t):
      sizeX = int(math.sqrt(t.shape[1])) #Размерность матрицы по X и Y
      return (np.dot(self.weights3, tanh(np.dot(self.weights2, prelu(np.dot(self.weights1, t) + self.bias_1, self.alpha)) + self.bias_2)) + self.b)


In [226]:
def avgErr(y_true, y_pred):
      return ((y_true - y_pred) ** 2).mean()

In [227]:
def func(x,y):
    return -((3 / (x*x + 8)) + (5 / (y*y + 8)))

In [228]:
dim_x = 100
#XY = np.random.uniform(-20, 20, 2*dim_x)
#XY.shape = (2,dim_x)
#X1 , Y1 = XY
X = np.arange(-20, 20, 1)
Y = np.arange(-20, 20, 1)
X, Y = np.meshgrid(X, Y)
X1, Y1 = X.ravel(), Y.ravel()
Z1 = func(X1, Y1)

dataset = np.array([X1, Y1])


In [229]:
Xd = np.array([X1,Y1])
Xd.shape

(2, 1600)

In [230]:
Zd = np.array([list(Z1)])
Zd.shape

(1, 1600)

In [231]:
z = NeuralNetwork_two(Xd,Zd)

# print(z.test(Xd))
# print(Z1)

z.alpha = 0.003
avg_pred = 0


In [ ]:
for i in range(10000000):
    z.feedforward()
    z.backprop()
 
    if i % 1000 == 0:
      avg = avgErr(Zd, z.test(Xd))
      if (np.absolute(avg_pred - avg) < 0.001) and z.alpha > 0.00001:
        z.alpha /= 2    
      avg_pred = avg
      print('avg:', avg)
    if avg < 0.0001:
      break



avg: 11.49547830752336
avg: 0.02400831375884663
avg: 0.028973218109933762
avg: 0.011257855171895344
avg: 0.010310252263486683
avg: 0.010082793642749657
avg: 0.009946485573843453
avg: 0.00987501056394018
avg: 0.009833881111418133
avg: 0.00981345740521255
avg: 0.00980269422637564
avg: 0.009798204471898594
avg: 0.009796173732948865
avg: 0.00979523235840044
avg: 0.009794268959640189
avg: 0.009793296091549539
avg: 0.00979232952883669
avg: 0.009791300035025189
avg: 0.009790180905381658
avg: 0.009789058113653106
avg: 0.009787905147513525
avg: 0.009786718979441665
avg: 0.009785497382828067
avg: 0.009784361805709102
avg: 0.009783198567826832
avg: 0.0097820036408469
avg: 0.009780779358692964
avg: 0.009779507310298956
avg: 0.00977818514795606
avg: 0.009776868108036591
avg: 0.009775658399328509
avg: 0.009774426130263091
avg: 0.009773169887351763
avg: 0.009771889635411696


In [ ]:
fig = plt.figure(figsize=plt.figaspect(0.5))

ax = fig.add_subplot(1, 2, 1, projection='3d')
X = np.arange(-20, 20, 0.25)
Y = np.arange(-20, 20, 0.25)
X, Y = np.meshgrid(X, Y)
Z = func(X, Y)
#X, Y, Z = get_test_data(0.05)
ax.plot_surface(X, Y, Z, rstride=1, cstride=1, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)
ax = fig.add_subplot(1, 2, 2, projection='3d')

ax.plot_wireframe(X, Y, Z, rstride=10, cstride=10)


In [ ]:
fig_2 = plt.figure(figsize=plt.figaspect(0.5))

ax_2 = fig_2.add_subplot(1, 2, 1, projection='3d')
X = np.arange(-20, 20.25, 0.25)
Y = np.arange(-20, 20.25, 0.25)
X, Y = np.meshgrid(X, Y)

Z = z.test(np.array([X.ravel(),Y.ravel()])).reshape((161,161))

ax_2.plot_surface(X, Y, Z, rstride=1, cstride=1, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)
ax_2 = fig_2.add_subplot(1, 2, 2, projection='3d')

ax_2.plot_wireframe(X, Y, Z, rstride=10, cstride=10)